In [63]:
# Importation des librairies

import numpy as np
import pandas as pd
import warnings

# Ignorer les warnings
warnings.filterwarnings('ignore')

# Fixer seed pour toujours avoir les mêmes valeurs aléatoires
np.random.seed(42)

features = ['ref_cli', 'prov', 'reg', 'com', 'fkt', 'date_dep_dmd', 'date_val_dmd',
            'date_int', 'tar', 'cat', 'pce']

cp_trn = np.arange(101, 120)
cp_tve = np.arange(501, 519)
cp_mjg = np.arange(401, 421)
cp_fnr = np.arange(301, 324)
cp_die = np.arange(202, 210)
cp_tul = np.arange(601, 622)

cp = dict()
cp['Antananarivo'] = cp_trn


# cp['Toamasina'] = cp_tve
# cp['Mahajanga'] = cp_mjg
# cp['Fianarantsoa'] = cp_fnr
# cp['Antsiranana'] = cp_die
# cp['Toliara'] = cp_tul


### Contrainte & conditions: 
    - Depuis 1980 à 2023
    - de janvier à décembre
    - Valeur aléatoire de 1 à 10 pour le nombre de demandes déposées pour chaque Fokontany
    - Saisonnalité 
    - Tendance progressive des données au fil du temps

In [64]:
# Importation de la liste des provinces, regions, districts, communes, fokontany de Madagascar
data_df = pd.read_csv('pro_reg_dist_com.csv', delimiter=';')
data_df = data_df.drop('N°', axis=1)

prod_conso_energy = pd.read_csv('scrap/prod_conso_energy.csv')
prod_conso_energy

,Date,Production GWh,Consommation GWh,Consommation par habitant kWh
0,1980,417.000,388.000,447
1,1981,420.000,391.000,438
2,1982,431.000,401.000,438
3,1983,452.000,420.000,447
4,1984,476.000,443.000,459
5,1985,483.000,449.000,454
6,1986,482.000,448.000,441
7,1987,478.000,445.000,426
8,1988,490.000,456.000,425
9,1989,541.000,503.000,457


In [65]:
prod_conso_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date                           42 non-null     int64  
 1   Production GWh                 42 non-null     float64
 2   Consommation GWh               42 non-null     float64
 3   Consommation par habitant kWh  42 non-null     int64  
dtypes: float64(2), int64(2)
memory usage: 1.4 KB


In [66]:
# Ajout 2 nouvelles observation pour l'année 2022, 2023
line1 = pd.DataFrame([[2022, 215.0, 200.92, 694]], columns=prod_conso_energy.columns)
line2 = pd.DataFrame([[2023, 230.0, 218.55, 940]], columns=prod_conso_energy.columns)
prod_conso_energy = pd.concat([prod_conso_energy, line1])
prod_conso_energy = pd.concat([prod_conso_energy, line2])
prod_conso_energy = prod_conso_energy.reset_index(drop=True)

# Correction des valeurs de 'Consommation par habitant kWh'
prod_conso_energy['Consommation par habitant kWh'] = prod_conso_energy['Consommation par habitant kWh'].apply(lambda x: x/10)

# Correction des valeurs de 'Consommation GWh'
prod_conso_energy['Consommation GWh'] = prod_conso_energy['Consommation GWh'].apply(lambda x: x/1000 if x > 3 else x)

# Correction des valeurs de 'Production GWh'
prod_conso_energy['Production GWh'] = prod_conso_energy['Production GWh'].apply(lambda y: y/1000 if y > 3 else y)


prod_conso_energy

,Date,Production GWh,Consommation GWh,Consommation par habitant kWh
0,1980,0.417,0.38800,44.7
1,1981,0.420,0.39100,43.8
2,1982,0.431,0.40100,43.8
3,1983,0.452,0.42000,44.7
4,1984,0.476,0.44300,45.9
5,1985,0.483,0.44900,45.4
6,1986,0.482,0.44800,44.1
7,1987,0.478,0.44500,42.6
8,1988,0.490,0.45600,42.5
9,1989,0.541,0.50300,45.7


In [67]:
# Nettoyage des données
df_copied = data_df.copy()

# Remplacement des valeurs '\\N' par null
df_copied.replace('\\N', np.nan, inplace=True)

# Remplacement des valeurs 'Unknown Name' par null
df_copied.replace('Unknown Name', np.nan, inplace=True)

# Suppression de toutes les lignes contenant les valeurs nulles
df_copied.dropna(inplace=True)

# Réinitialistion de l'index après suppression
fkt_df = df_copied.reset_index(drop=True)
fkt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17121 entries, 0 to 17120
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fokontany  17121 non-null  object
 1   Kaomina    17121 non-null  object
 2   Distrika   17121 non-null  object
 3   Region     17121 non-null  object
 4   Province   17121 non-null  object
dtypes: object(5)
memory usage: 668.9+ KB


D'après les informations affichées, il y a 17121 observations dans le data, lesquelles sont organisées en Provinces, Regions, Districts, Communes et Fokontany ou quartier.  
Mais ce qui nous interesse c'est de récupérer seulement la liste des districts de la région d'Analamanga de la province d'Antananarivo. L'idée c'est de travailler sur un échantillon un peu réduit pour éviter les calculs trop important.

In [68]:
# Filtre par Région d'Analamanga
analamanga_df = fkt_df[fkt_df['Region'] == 'Analamanga']
analamanga_df.reset_index(drop=True)

# Récupération de la liste de districts
districts = analamanga_df['Distrika'].unique()
districts

array(['Andramasina', 'Antananarivo-Avaradrano',
       'Antananarivo-Atsimondrano', 'Manjakandriana', 'Ambohidratrimo',
       'Anjozorobe', 'Ankazobe'], dtype=object)

In [69]:
# index_sep = prod_conso_energy[prod_conso_energy['Date'] == 2000].index[0]
# index_sep

# prod_conso_energy = prod_conso_energy[index_sep:]
# prod_conso_energy.reset_index(drop=True)
# prod_conso_energy.info()

In [70]:
# Création des données
years = range(1980, 2024)

data = {'district': [],
        'nb_dmd': [],
        'energy': [],
        'date_dep_dmd': []}

for year in years:
    # conso_mean_monthly = [prod_conso_energy['Consommation GWh'][prod_conso_energy.index == year]] / 12
    row = prod_conso_energy.loc[prod_conso_energy['Date'] == year]
    conso_mean_monthly = row.loc[row.index[0],'Consommation par habitant kWh'] / 12
    conso_mean_global = row.loc[row.index[0],'Consommation GWh'] / 12
    for month in range(1, 13):
        for district in districts:
            # Génération de la demande en électricité avec une saisonnalité et une tendance progressive
            demand = conso_mean_global + 10 * (year - 1980) + np.sin(month * np.pi / 6) * 20
            demand = int(demand)
            # Ajout d'une variation chaque année
            demand += np.random.randint(-20, 20)
            
            # Génération de la puissance énergétique avec une saisonnalité et une tendance progressive
            power = (conso_mean_monthly * demand) / 1_000_000
            
            data['district'].append(district)
            data['nb_dmd'].append(demand)
            data['energy'].append(power)
            data['date_dep_dmd'].append(pd.Timestamp(year, month, 1))

# Création du dataframe
df = pd.DataFrame(data)


In [75]:
# Modification du nom de la colonne
df.rename(columns={'date_dep_dmd':'date'}, inplace=True)
df[df['nb_dmd'] < 0]

,district,nb_dmd,energy,date
3,Manjakandriana,-3,-0.000011,1980-01-01
30,Antananarivo-Atsimondrano,-8,-0.000030,1980-05-01
32,Ambohidratrimo,-4,-0.000015,1980-05-01
34,Ankazobe,-2,-0.000007,1980-05-01
36,Antananarivo-Avaradrano,-3,-0.000011,1980-06-01
...,...,...,...,...
239,Antananarivo-Avaradrano,-8,-0.000029,1982-11-01
307,Ankazobe,-6,-0.000022,1983-08-01
311,Manjakandriana,-8,-0.000030,1983-09-01
317,Antananarivo-Atsimondrano,-7,-0.000026,1983-10-01


**date_dep_dmd**: c'est la date de dépôt de la demande effectué par un potentiel client de la JIRAMA.

**district**: Nom des disctricts de la région d'Analamanga

**nb_dmd**: Nombre de demandes déposées

**energy**: Puissance minimal en energie électrique requise, en Mega Watt.

On sauvegarde les données aux format 'csv' pour une utilisation ultérieure

In [72]:
# Sauvegarde de la collection de données
df.to_csv('energy_antananarivo_collected.csv', index=False)